In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
import os
import shutil
from glob import glob

In [ ]:
### TODO 1: Split the training data into folders train and test with subfolders for each class
### Additionally, also convert the data (normalise the pixel values)
data_dir = 'data'
train_dir = 'data/train'
test_dir = 'data/test'

# Create train and test directories if not already created
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Split images into train and test folders
for class_name in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, class_name)):
        class_path = os.path.join(data_dir, class_name)
        images = glob(os.path.join(class_path, '*.jpg'))
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
        
        class_train_dir = os.path.join(train_dir, class_name)
        class_test_dir = os.path.join(test_dir, class_name)

        os.makedirs(class_train_dir, exist_ok=True)
        os.makedirs(class_test_dir, exist_ok=True)

        for img in train_images:
            shutil.copy(img, class_train_dir)
        for img in test_images:
            shutil.copy(img, class_test_dir)

# Normalize pixel values and apply data augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, width_shift_range=0.2,
                                    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                    horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust as per the dataset
    batch_size=32,
    class_mode='binary'  # Use 'binary' for binary classification or 'categorical' for multi-class
)

test_data = val_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
input_shape = (150, 150, 3) # Check dimensions of the images
model = Sequential([
    InputLayer(input_shape=input_shape),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') # For binary classification; use 'softmax' with more neurons for multi-class
])

In [ ]:
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])
# TODO 3: Modify train_data and the value of epochs accordingly
model.fit(train_data, epochs=10, validation_data=test_data)  # Adjust epochs based on the dataset size and complexity

In [ ]:
test_loss, test_acc = model.evaluate(test_data, steps=test_data.samples // test_data.batch_size)
print(f'Test accuracy: {test_acc}')